In [37]:
import pandas as pd
import numpy as np
from numba import njit
import vectorbtpro as vbt

In [38]:
data = vbt.BinanceData.fetch(["BTCUSDT", "ETHUSDT"])

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [39]:
sub_data = data.loc["2021-02-18":"2021-02-24"]
vbt.settings.set_theme("dark")
sub_data.plot(symbol="BTCUSDT")

FigureWidget({
    'data': [{'close': array([51552.6 , 55906.  , 55841.19, 57408.57, 54087.67, 48891.  , 49676…

This is data pertaining to 18th Feb 2021 to 24 Feb 2021

In [40]:
X = True
O = False

In [41]:
entries = pd.Series([X, O, O, O, O, O, O])
exits   = pd.Series([O, O, O, X, O, O, O])

In [42]:
pf = vbt.Portfolio.from_signals(
     sub_data,
     entries=entries,  
     exits=exits
 )

In [43]:
pf.orders.records_readable

,Order Id,Column,Signal Index,Creation Index,Fill Index,Size,Price,Fees,Side,Type,Stop Type
0,0,BTCUSDT,2021-02-18 00:00:00+00:00,2021-02-18 00:00:00+00:00,2021-02-18 00:00:00+00:00,0.001940,51552.60,0.0,Buy,Market,None
1,1,BTCUSDT,2021-02-21 00:00:00+00:00,2021-02-21 00:00:00+00:00,2021-02-21 00:00:00+00:00,0.001940,57408.57,0.0,Sell,Market,None
2,0,ETHUSDT,2021-02-18 00:00:00+00:00,2021-02-18 00:00:00+00:00,2021-02-18 00:00:00+00:00,0.051557,1939.61,0.0,Buy,Market,None
3,1,ETHUSDT,2021-02-21 00:00:00+00:00,2021-02-21 00:00:00+00:00,2021-02-21 00:00:00+00:00,0.051557,1933.53,0.0,Sell,Market,None


In [44]:
pf.assets

symbol,BTCUSDT,ETHUSDT
Open time,,
2021-02-18 00:00:00+00:00,0.00194,0.051557
2021-02-19 00:00:00+00:00,0.00194,0.051557
2021-02-20 00:00:00+00:00,0.00194,0.051557
2021-02-21 00:00:00+00:00,0.00000,0.000000
2021-02-22 00:00:00+00:00,0.00000,0.000000
2021-02-23 00:00:00+00:00,0.00000,0.000000
2021-02-24 00:00:00+00:00,0.00000,0.000000


In [50]:
@njit(debug=True)
def signal_func_nb(c, entries, exits, short_entries, short_exits, size = None):
     long_entry = entries[c.i]
     long_exit = exits[c.i]
     short_entry = short_entries[c.i]
     short_exit = short_exits[c.i]
     print(f"Current Row: {c.i} | Current Column: {c.col}")
     print(f"LongEntry: {str(long_entry).upper()} | LongExit: {str(long_exit).upper()} | \
             ShortEntry: {str(short_entry).upper()} | ShortExit: {str(short_exit).upper()}\
             Size : {c.col} ")
     return long_entry, long_exit, short_entry, short_exit

In [51]:
%%time
pf = vbt.Portfolio.from_signals(
     sub_data.select("BTCUSDT"),
     signal_func_nb=signal_func_nb,
     signal_args=(
         pd.Series([X, O, O, O, O, O, O]).values,
         pd.Series([O, X, O, X, O, O, O]).values,
         pd.Series([O, O, O, O, X, O, O]).values,
         pd.Series([O, O, O, O, O, O, X]).values
     )
 )

/opt/miniconda3/envs/vbt/lib/python3.10/site-packages/vectorbtpro/portfolio/nb/from_signals.py:2678: NumbaDebugInfoWarning:

Could not find source for function: <function signal_func_nb at 0x29ed053f0>. Debug line information may be inaccurate.



Current Row: 0 | Current Column: 0
LongEntry: TRUE | LongExit: FALSE |              ShortEntry: FALSE | ShortExit: FALSE             Size : 0 
Current Row: 1 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE |              ShortEntry: FALSE | ShortExit: FALSE             Size : 0 
Current Row: 2 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE |              ShortEntry: FALSE | ShortExit: FALSE             Size : 0 
Current Row: 3 | Current Column: 0
LongEntry: FALSE | LongExit: TRUE |              ShortEntry: FALSE | ShortExit: FALSE             Size : 0 
Current Row: 4 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE |              ShortEntry: TRUE | ShortExit: FALSE             Size : 0 
Current Row: 5 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE |              ShortEntry: FALSE | ShortExit: FALSE             Size : 0 
Current Row: 6 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE |              ShortEntry: FALSE | ShortExit: TRUE             Size :

Using `Select Nb`

In [52]:
@njit
def signal_func_nb(c, entries, exits, short_entries, short_exits):
     long_entry = vbt.pf_nb.select_nb(c, entries)
     long_exit = vbt.pf_nb.select_nb(c, exits)
     short_entry = vbt.pf_nb.select_nb(c, short_entries)
     short_exit = vbt.pf_nb.select_nb(c, short_exits)
     print(f"Current Row: {c.i} | Current Column: {c.col}")
     print(f"LongEntry: {str(long_entry).upper()} | LongExit: {str(long_exit).upper()} | ShortEntry: {str(short_entry).upper()} | ShortExit: {str(short_exit).upper()}")
     return long_entry, long_exit, short_entry, short_exit

In [53]:
%%time
pf = vbt.Portfolio.from_signals(
     sub_data.select("BTCUSDT"),
     signal_func_nb=signal_func_nb,
     signal_args=(
         pd.Series([X, O, O, O, O, O, O]).values,
         pd.Series([O, O, O, X, O, O, O]).values,
         pd.Series([O, O, O, O, X, O, O]).values,
         pd.Series([O, O, O, O, O, O, X]).values,
         pd.Series([1.0, -0.5, 0.0, 0.5, 1.0, 0.0, 1.0 ]).values,
     )
 )

Current Row: 0 | Current Column: 0
LongEntry: TRUE | LongExit: FALSE | ShortEntry: FALSE | ShortExit: FALSE
Current Row: 1 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE | ShortEntry: FALSE | ShortExit: FALSE
Current Row: 2 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE | ShortEntry: FALSE | ShortExit: FALSE
Current Row: 3 | Current Column: 0
LongEntry: FALSE | LongExit: TRUE | ShortEntry: FALSE | ShortExit: FALSE
Current Row: 4 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE | ShortEntry: TRUE | ShortExit: FALSE
Current Row: 5 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE | ShortEntry: FALSE | ShortExit: FALSE
Current Row: 6 | Current Column: 0
LongEntry: FALSE | LongExit: FALSE | ShortEntry: FALSE | ShortExit: TRUE
CPU times: user 13.1 s, sys: 204 ms, total: 13.3 s
Wall time: 13.3 s
